# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [67]:
!ls

categories.csv			   ML Pipeline Preparation.ipynb
DisasterResponse.db		   ML Pipeline Preparation-zh.ipynb
ETL Pipeline Preparation.ipynb	   Twitter-sentiment-self-drive-DFE.csv
ETL Pipeline Preparation-zh.ipynb  Untitled1.ipynb
InsertDatabaseName.db		   Untitled.ipynb
messages.csv


In [68]:
# import libraries

import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
import re

from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import classification_report, f1_score, recall_score, precision_score, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin

try:
    from sklearn.model_selection import GridSearchCV
except : 
    print("old version sklearn detected..")
    from sklearn.grid_search import GridSearchCV


from sqlalchemy import create_engine

from nltk import word_tokenize
from nltk import sent_tokenize
from nltk.tag import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [69]:
# load data from database
engine = create_engine('sqlite:///InsertDatabaseName.db')

df = pd.read_sql_table(table_name="InsertTableName", con=engine)

X = df["message"]
y = df.drop(columns=["id", "message", "original", "genre"], axis=1)

X.head()

0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
2                      Looking for someone but no name
3    UN reports Leogane 80-90 destroyed. Only Hospi...
4    says: west side of Haiti, rest of the country ...
Name: message, dtype: object

### 2. Write a tokenization function to process your text data

In [70]:
def tokenize(text):

    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    
    # replace all urls with a placeholder text
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    urls = re.findall(url_regex, text)
    for url in urls : 
        text = text.replace(url, "urlplaceholder")
    
    stop_words = stopwords.words("english")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    # lemmatize andremove stop words
    clean_tokens = []
    for tok in tokens:
        if tok not in stop_words:
            clean_tok = lemmatizer.lemmatize(tok).strip() 
            clean_tokens.append(clean_tok)
    
    return clean_tokens


class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Select a single column from the data frame to perform additional transformations on
    Use on text columns in the data
    """
    def __init__(self, col):
        self.col = col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.col]


class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    """Returns true if sentence is found to 
    start with a verb.
    """
    
    def starting_verb(self, text):
        
        sentence_list = nltk.sent_tokenize(text)
        
        for sentence in sentence_list:
            
            pos_tags = nltk.pos_tag(tokenize(sentence))
            
            try:
                first_word, first_tag = pos_tags[0]
                if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                    return True
            except :
                continue
                
        return False
    
    def fit(self, x, y=None):
        return self
        
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [71]:
# need to add the filtering of the "message" column only here I guess ...

# pipe = Pipeline([
#     ('features', FeatureUnion([
#         ('text_pipeline', Pipeline([
#             ('col', ColumnSelector(col="message")),
#             ('vect', CountVectorizer(tokenizer=tokenize)),
#             ('tdidf', TfidfTransformer()),
#         ])),
# #         ('starting_verb', StartingVerbExtractor())
#     ])),
#     ('clf', MultiOutputClassifier(RandomForestClassifier()))
# ])

pipe = Pipeline([
#     ('col', ColumnSelector(col="message")),
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tdidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [72]:
# split and fit data
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
%%time
pipe.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
# predict on test data
def evaluate(X_test, y_test):
    """Predicts and prints scores of model.
    """
    
    y_pred = pipe.predict(X_test)
    
    clf_report = classification_report(y_test, y_pred, target_names=y_test.columns, output_dict=False)
    clf_report_dict = classification_report(y_test, y_pred, target_names=y_test.columns, output_dict=True)
    
    print(clf_report)
    
    try :
        df = pd.DataFrame.from_dict(clf_report_dict, orient="index")

        plt.figure(figsize=(6,6))
        sns.barplot(df["f1-score"].sort_values(), df["f1-score"].sort_values().index)
    
    except : 
        print("Error building graph.")
        continue
        
        
    return clf_report_dict


evaluate(X_test, y_test)

SyntaxError: 'continue' not properly in loop (<ipython-input-35-61b80be17f42>, line 21)

### 6. Improve your model
Use grid search to find better parameters. 

In [41]:
%%time
parameters = {
    'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
    'features__text_pipeline__vect__max_df': (0.5, 0.75, 1.0),
    'features__text_pipeline__vect__max_features': (None, 5000, 10000),
}

cv = GridSearchCV(pipe, param_grid=parameters)

cv.fit(X_train["message"].values, y_train)

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = cv.predict(X_test["message"].values)

display_results(y_test, y_pred)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [ ]:
pickle.dump(model, open( model_filepath, "wb" ))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.